In [1]:
from src.agents import Volatile

In [2]:
N_MOLECULE = 100000
volatiles = Volatile()

In [ ]:

for j in range(5000):
    volatiles.migrate(2.989e-26)

In [7]:
print(sum(volatiles.cold))
print(sum(volatiles.jeans))
print(sum(volatiles.photo))
print(max(volatiles.velocity))

3180
0
0
4159.504767242616
